In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tifffile
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm
import joblib as jb
from glob import glob

In [19]:
def get_coord_from_pixel_pos(i, j, start_value_x, start_value_y, scale_x, scale_y):
    """
    i : int - coord x 
    j : int - coord y
    start_value_x : int - start coord x UTM
    start_value_y : int - start coord y UTM
    scale_x : int - pixel/meter scale x
    scale_y : int - pixel/meter scale y
    """
    return (start_value_x+scale_x*i, start_value_y-scale_y*j)

def get_pixel_from_coord(coord_x,coord_y, scale_x, scale_y, start_x, start_y):
    return (int((coord_x-start_x)/scale_x), int((start_y-coord_y)/scale_y))

In [17]:
path = Path('../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_560.tif')
path_bat = '/home/vinicius/Área de Trabalho/Faculdade/TCC/data/DADOS_TCC_BIANCA/02.Dados_Processados/02.XYZ/BATIMETRIA_PROCESSADA_BABITONGA_2018_DHN_WGS84_UTM22S.xyz'

In [11]:
paths_list = [e for e in glob('../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/*.tif')]
rhos_list = [e for e in glob('../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/*.tif') if 'rhos' in str(e)]
rhow_list = [e for e in glob('../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/*.tif') if 'rhow' in str(e)]

In [13]:
tiff_array = [tifffile.TiffFile(path).asarray() for path in rhos_list]

In [22]:
tiff = tifffile.TiffFile(rhos_list[0])
aux_dict = dict()
for page in tiff.pages:
    for tag in page.tags:
        aux_dict[tag.name] = tag.value

scale_x = aux_dict['ModelPixelScaleTag'][0]
scale_y = aux_dict['ModelPixelScaleTag'][1]
start_value_x = aux_dict['ModelTiepointTag'][3]
start_value_y = aux_dict['ModelTiepointTag'][4]

In [18]:
df = pd.read_csv(path_bat, sep=' ', header=None)
df.columns = ['x', 'y', 'z']
df

,x,y,z
0,720507.82,7089857.17,1.96
1,720507.95,7089857.15,1.98
2,720508.07,7089857.12,1.98
3,720508.19,7089857.09,1.97
4,720508.31,7089857.07,1.98
...,...,...,...
557239,720868.85,7107250.49,3.41
557240,720868.95,7107250.55,3.41
557241,720869.05,7107250.60,3.41
557242,720869.14,7107250.65,3.41


In [29]:
rhos_list

['../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_740.tif',
 '../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_704.tif',
 '../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_665.tif',
 '../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_492.tif',
 '../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_560.tif',
 '../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_783.tif',
 '../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_833.tif',
 '../data/DADOS_TCC_BIANCA/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_443.tif']

In [32]:
for i, band in enumerate(tiff_array):
    column_name = 'reflectance_{}'.format(i)
    aux = list()
    for i, row in enumerate(df.itertuples()):
        x = row[1]
        y = row[2]
        z = row[3]
        pixel = get_pixel_from_coord(x, y, scale_x, scale_y, start_value_x, start_value_y)
        aux.append(band[pixel[1]][pixel[0]])
        
    df[column_name] = aux
    
        


# Modelagem

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import r2_score

In [46]:
features = df.drop(['z', 'y', 'x'], axis=1)
target = df['z']

In [50]:
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.5)

In [51]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((278622, 8), (278622,), (278622, 8), (278622,))

In [52]:
lgbm = LGBMRegressor(n_estimators=500)

In [53]:
lgbm.fit(X_train, y_train)

LGBMRegressor(n_estimators=500)

In [54]:
p = lgbm.predict(X_val)

In [59]:
X_val

,reflectance_0,reflectance_1,reflectance_2,reflectance_3,reflectance_4,reflectance_5,reflectance_6,reflectance_7
288507,0.007875,0.017208,0.021635,0.024913,0.038754,0.277486,0.007292,0.018943
504512,0.009522,0.020321,0.020550,0.021204,0.026869,0.271536,0.007672,0.011705
154117,0.008803,0.017829,0.021037,0.027923,0.038518,0.238308,0.008444,0.015036
26675,0.008799,0.017243,0.020025,0.014252,0.026632,0.284374,0.008155,0.007908
171836,0.011007,0.026654,0.030619,0.029266,0.043253,0.282078,0.010587,0.021316
...,...,...,...,...,...,...,...,...
117269,0.014159,0.038710,0.045437,0.041981,0.059104,0.241968,0.010885,0.027318
234838,0.011234,0.024619,0.029623,0.028309,0.044243,0.324191,0.011015,0.020663
426232,0.008743,0.016202,0.018655,0.025156,0.034405,0.221464,0.008423,0.014978
141622,0.012180,0.026751,0.031839,0.029498,0.045755,0.284791,0.010117,0.020433


In [55]:
r2_score(y_val, p)

0.8979660065989385

In [56]:
space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 10), # max_depth
          (1, 20), # min_child_samples
          (0.05, 1.), # subsample
          (0.05, 1.), # colsample_bytree
          (100,1000)] # n_estimator


In [57]:
from skopt import forest_minimize

def tune_lgbm(args):
    lr = args[0]
    max_depth = args[1]
    min_child_samples = args[2]
    subsample = args[3]
    colsample_bytree = args[4]
    n_estimators = args[5]
    
    mdl = LGBMRegressor(learning_rate=lr, max_depth=max_depth, min_child_samples=min_child_samples, subsample=subsample, colsample_bytree=colsample_bytree, n_estimators=n_estimators)
    mdl.fit(X_train, y_train)
    p = mdl.predict(X_val)
    print(r2_score(y_val, p))
    
    return -r2_score(y_val, p)

In [58]:
res = forest_minimize(tune_lgbm, space, random_state=160000, n_random_starts=20, n_calls=50, verbose=0)

0.5810931121243449
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
0.7501886657340556
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
0.6987614913331419
0.3825602803310234
0.4408767919951665
0.6560283926767049
0.543215864402514
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
0.6556890787174623
0.45284927788830265
0.5827639564091305
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
0.0793886732331871
0.546689947353783
0.6667603149057999
0.6134208435778127
0.7425502205574497
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
0.5943114138931671
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
0.8514635080223554
0.6405815141359533
0.25830053481083515
[LightGBM] [Warning] Accura

In [60]:
# melhores hiperparametros para o modelo baseado em bayesian optimization
res['x']

[0.09740262475085597, 8, 19, 0.2518783353639487, 0.9892916499919282, 987]

In [ ]:
lr = args[0]
max_depth = args[1]
min_child_samples = args[2]
subsample = args[3]
colsample_bytree = args[4]
n_estimators = args[5]

In [69]:
hyper = {
    'lr': res['x'][0],
    "max_depth": res['x'][1],
    "min_child_samples": res['x'][2],
    "subsample": res['x'][3],
    'colsample_bytree': res['x'][4],
    'n_estimators': res['x'][5]
}

In [70]:
mdl = LGBMRegressor(learning_rate=hyper['lr'], max_depth=hyper['max_depth'], 
                    min_child_samples=hyper['min_child_samples'], subsample=hyper['subsample'], 
                    colsample_bytree=hyper['colsample_bytree'], n_estimators=hyper['n_estimators'])

In [75]:
mdl.fit(X_train, y_train)

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


LGBMRegressor(colsample_bytree=0.9892916499919282,
              learning_rate=0.09740262475085597, max_depth=8,
              min_child_samples=19, n_estimators=987,
              subsample=0.2518783353639487)

In [76]:
p = mdl.predict(X_val)

In [77]:
r2_score(y_val, p)

0.9401299647792786

In [81]:
import joblib as jb

In [82]:
jb.dump(mdl, 'models/lgbm_baseline.pkl.z')

['models/lgbm_baseline.pkl.z']

# 2.0 - Validação Cruzada

In [111]:
from sklearn.model_selection import KFold

def cross_validation():
    global hyper
    k_fold_splits = {}
    kf = KFold(n_splits=10, shuffle=True)
    for i, (lines_train, lines_val) in enumerate(kf.split(features)):
        X_train = features.iloc[lines_train]
        X_val = features.iloc[lines_val]
        y_train = target.iloc[lines_train]
        y_val = target.iloc[lines_val]
        
        mdl = LGBMRegressor(learning_rate=hyper['lr'], max_depth=hyper['max_depth'], 
                    min_child_samples=hyper['min_child_samples'], subsample=hyper['subsample'], 
                    colsample_bytree=hyper['colsample_bytree'], n_estimators=hyper['n_estimators'])
        mdl.fit(X_train, y_train)
        p = mdl.predict(X_val)
        score = r2_score(y_val, p)
        
        k_fold_splits[f'k_fold_{i}'] = score
        
    return k_fold_splits

In [ ]:
scores = cross_validation()

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


In [98]:
scores

{'k_fold_0': 0.9412270778569117,
 'k_fold_1': 0.9398243693733003,
 'k_fold_2': 0.9402682015328211,
 'k_fold_3': 0.9398492633931795,
 'k_fold_4': 0.9405965944655289}